# This File is Done in DataBricks

1. Create a Cluster
2. import dataset
3. create notebook and attach with cluster

In [ ]:
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

df = spark.read.csv(file_location,header=True,inferSchema=True)
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [ ]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [ ]:
df.columns

Out[4]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [ ]:
## Handling Categorical Features
from pyspark.ml.feature import StringIndexer


In [ ]:
indexer = StringIndexer(inputCols=["sex","day","smoker","time"],outputCols=["sex_indexed","day_indexed","smoker_indexed","time_indexed"])
df_r=indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+-----------+--------------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|day_indexed|smoker_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+-----------+--------------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|        1.0|           0.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|        1.0|           0.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|        1.0|           0.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|        1.0|           0.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|        1.0|           0.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|        1.0|           0.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|        1.0|           0.0|         0.0|


In [ ]:
df_r.columns

Out[15]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'day_indexed',
 'smoker_indexed',
 'time_indexed']

In [ ]:
from pyspark.ml.feature import VectorAssembler
featureAssembler = VectorAssembler(inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed'],
                outputCol='Independent Feature')
output=featureAssembler.transform(df_r)

In [ ]:
output.select('Independent Feature').show()

+--------------------+
| Independent Feature|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(5,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [ ]:
finalized_data = output.select('Independent Feature','total_bill')

In [ ]:
finalized_data.show()

+--------------------+----------+
| Independent Feature|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(5,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [ ]:
from pyspark.ml.regression import LinearRegression
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Feature',labelCol='total_bill')
regressor=regressor.fit(train_data)

In [ ]:
regressor.coefficients

Out[26]: DenseVector([3.1065, 3.3252, -1.052, 2.5945, -0.6745])

In [ ]:
regressor.intercept

Out[27]: 2.368140010152908

In [ ]:
pred_results = regressor.evaluate(test_data)

In [ ]:
pred_results.predictions.show()

+--------------------+----------+------------------+
| Independent Feature|total_bill|        prediction|
+--------------------+----------+------------------+
|(5,[0,1],[1.45,2.0])|      9.55|13.522986724590542|
|(5,[0,1],[1.75,2.0])|     17.82| 14.45492537064864|
|(5,[0,1],[2.01,2.0])|     20.23| 15.26260553056566|
|(5,[0,1],[2.31,3.0])|     18.69|19.519782472535503|
|(5,[0,1],[2.34,4.0])|     17.81|22.938214633053057|
|(5,[0,1],[2.72,2.0])|     13.28|17.468193659569828|
|(5,[0,1],[3.15,3.0])|     20.08| 22.12921068149818|
|(5,[0,1],[3.27,2.0])|     17.78|19.176747844009675|
|(5,[0,1],[4.08,2.0])|     17.92|21.692982188366543|
|(5,[0,1],[5.92,3.0])|     29.03|30.734110846767962|
|[1.0,1.0,1.0,0.0,...|      7.25| 7.747835420333638|
|[1.0,1.0,1.0,1.0,...|      3.07|10.342289644390778|
|[1.1,2.0,1.0,1.0,...|      12.9|13.978174155655221|
|[1.25,2.0,1.0,0.0...|      8.51|10.500629588733128|
|[1.44,2.0,0.0,0.0...|      7.56|12.142862437161268|
|[1.5,2.0,0.0,1.0,...|     11.59|16.2727640563

In [ ]:
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError

Out[30]: (0.6503890810809748, 3.647832811494077, 22.339664130966735)